In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
import lightgbm as lgb
from dateutil.parser import parse
import matplotlib.pyplot as plt
import seaborn as sns
#from matplotlib.pylab import rcParams
import time
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split,GridSearchCV
# rcParams['figure.figsize'] = 12, 8
#rcParams['figure.dpi'] = 300

In [2]:
train_data = pd.read_csv("round1_ijcai_18_train_20180301.txt", sep=" ")
test_data = pd.read_csv("round1_ijcai_18_test_a_20180301.txt", sep=" ")
#print(train_data.head())
#看是否有缺失值
train_data.drop_duplicates(inplace=True)#默认为所有行相同去重，也可以指定列下去重
train_data.isnull().any().any()
test_data.isnull().any().any()
#训练数据--测试数据 是否信息类目一致
#diff = train_columns.difference(test_cloumns)
#print(diff)
train_columns = set(train_data.columns)
#print(train_columns)
test_cloumns = set(test_data.columns)
diff = train_columns - test_cloumns
#是否有字符串，有可以做LabelEncode

train_data.describe()
#负样本8994 << 正样本469117 =50倍(样本不平衡问题)https://www.leiphone.com/news/201706/dTRE5ow9qBVLkZSY.html

#print(train_data['is_trade'].value_counts())
#print(train_data.loc[:,'is_trade'].value_counts()
#统计信息绘图
# dis = train_data['is_trade'].value_counts()
# plt.figure(figsize = (20, 8))
# sns.barplot(dis.index, dis.values, alpha=0.7, color='red')
# plt.xlabel('value', fontsize=10)
# plt.ylabel('number', fontsize=10)
# plt.show()
# print('hello')

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,context_timestamp,context_page_id,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
count,4.781110e+05,4.781110e+05,4.781110e+05,4.781110e+05,478111.000000,478111.000000,478111.000000,478111.000000,4.781110e+05,478111.000000,...,4.781110e+05,478111.000000,4.781110e+05,478111.000000,478111.000000,478111.000000,478111.000000,478111.000000,478111.000000,478111.000000
mean,4.613058e+18,4.594616e+18,4.838454e+18,5.500479e+18,6.777242,11.133741,12.199288,17.128648,4.609870e+18,0.213045,...,1.537502e+09,4003.362343,4.733809e+18,15.768296,0.994830,5013.175187,0.971124,0.970497,0.974863,0.018812
std,2.663453e+18,2.710124e+18,2.694774e+18,2.464859e+18,1.088979,2.588877,2.497317,2.160771,2.663365e+18,0.514703,...,1.726216e+05,4.019755,2.596096e+18,2.825281,0.011972,2.590979,0.023818,0.023777,0.025025,0.135859
min,3.823219e+12,6.964907e+14,-1.000000e+00,-1.000000e+00,0.000000,-1.000000,0.000000,0.000000,2.477979e+13,-1.000000,...,1.537200e+09,4001.000000,1.543560e+15,0.000000,-1.000000,4999.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,2.308539e+18,2.251088e+18,2.357806e+18,3.948283e+18,6.000000,10.000000,11.000000,16.000000,2.302746e+18,0.000000,...,1.537351e+09,4001.000000,2.498543e+18,14.000000,0.992779,5012.000000,0.966360,0.965684,0.969268,0.000000
50%,4.613362e+18,4.636270e+18,5.051040e+18,6.738284e+18,7.000000,11.000000,12.000000,17.000000,4.608564e+18,0.000000,...,1.537506e+09,4001.000000,4.629590e+18,16.000000,1.000000,5013.000000,0.972347,0.971590,0.978493,0.000000
75%,6.920942e+18,7.007158e+18,7.565625e+18,7.534239e+18,8.000000,13.000000,14.000000,19.000000,6.914401e+18,0.000000,...,1.537635e+09,4004.000000,6.740687e+18,17.000000,1.000000,5015.000000,0.977822,0.976978,0.983640,0.000000
max,9.223362e+18,9.221980e+18,9.222396e+18,9.151271e+18,17.000000,17.000000,17.000000,21.000000,9.223345e+18,2.000000,...,1.537805e+09,4020.000000,9.220773e+18,25.000000,1.000000,5020.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
def timestamp_convert(value):
    format = '%Y-%m-%d %H:%M:%S'#要转化的格式
    value = time.localtime(value)
    #print(value)
    newday = time.strftime(format, value)
    return newday

def covert_seconds_to_days(data):
    #frame['days'] = (pd.to_datetime(frame['date']) - parse('2015-01-01')).dt.days#转化为时间
    data['date'] = data['context_timestamp'].apply(timestamp_convert)#将这一列的每个数据进行一一转换
    data['day'] = data.date.apply(lambda x: int(x[8:10]))#将这一列的每个数据进行一一切片
    data['hour'] = data.date.apply(lambda x: int(x[11:13]))
#     user_query_day = data.groupby(['user_id', 'day']).size().reset_index().rename(columns={0:'user_query_day'})
#      #默认inner。left、right分别保留左边、右边部分数据（缺失补NAN）,inner公共部分,outer俩边所有
#     data = pd.merge(data, user_query_day, how='inner', on=['user_id','user_query_day'])
#     user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index().rename(columns={1:"user_query_day_hour"})
#     data = pd.merge(data, user_query_day_hour, how='inner', on=['user_id', 'day', 'hour'])
    del data['context_timestamp']
    return data

def lgb_baselinemodel(train_data,test_data,online):
    features = ['context_id',
             'context_page_id',
             'instance_id',         
             'item_brand_id',
             'item_city_id',
             'item_collected_level',
             'item_id',
             'item_price_level',
             'item_pv_level',
             'shop_id',
             'shop_review_positive_rate',
             'shop_score_delivery',
             'shop_score_description',
             'shop_star_level',
             'user_age_level',
             'user_gender_id',
             'user_id',
             'day',
             'hour']
    labels = 'is_trade'
    if online == False:
        training_data = train_data[(train_data['day'] < 24) & (train_data['day'] >= 18)]
        #print(train_data)
        train_test_data = train_data[train_data['day'] == 24]
#        print(train_test_data)
        #feature_test = train_test_data.columns.tolist().remove('is_trade')
        #drop默认删除行axis=0，返回一个新的DF可以不改变原有数据，inplace=True时.drop()执行内部删除，不返回任何值，原数据发生改变
        #lgb_cv(train_data.drop['label', axis=1， inplace=True], train_data.iloc[:, ['label']])
        clf = lgb.LGBMClassifier(num_leaves=64, max_depth=8, n_estimators=100, n_jobs=20)
        clf.fit(training_data[features], training_data[labels], feature_name=features, categorical_feature=['user_gender_id'])
        train_test_data['pred'] = clf.predict_proba(train_test_data[features],)[:, 1]
#        print(train_test_data['pred'])
        logloss = log_loss(train_test_data['is_trade'],train_test_data['pred'])
        print(logloss)
        
    if online == True:
        train_data = train_data.copy()
        test_data = pd.read_csv("round1_ijcai_18_test_a_20180301.txt", sep=" ")
        test_data.drop_duplicates(inplace=True)
        test_data = covert_seconds_to_days(test_data)
        model = lgb.LGBMClassifier(num_leaves=64, max_depth=7, n_estimators=80, n_jobs=20)
        model.fit(train_data[features], train_data[labels], feature_name=features, categorical_feature=['user_gender_id'])
        test_data['proba'] = model.predict_proba(test_data[features],)[:, 1]
        test_data[['instance_id','proba']].to_csv('baseline1.csv', index=False, sep=" ")

def lgb_gridsearch_model(train_data):
    train_data.drop_duplicates(inplace=True)#默认为所有行相同去重，也可以指定列下去重
    train_data = covert_seconds_to_days(train_data)

    d_x = train_data.loc[:,features]
    d_y = train_data.loc[:,label]
#     train_X, test_X, train_y, test_y = train_test_split(
#             d_x, d_y, testsize = 0.3, random_state = 20)
    print ("create classifier ...")
    param_grid = {
        "learing rate": [0.05, 0.06, 0.07, 0.08, 0.1],
        "n_estimators": [50, 60, 70],
        "max_depth": [5, 6, 7, 8]
    }
    params = {
        "objective": 'binary',#二分类;multiclass多分类
        "metric": 'multi_logloss',
        "max_bin": 255,
        "max_depth": 7,
        "learning_rate": 0.25,
        "n_estimators": 80
    }
    model = lgb.LGBMClassifier(
        boosting_type='gbdt', objective='binary', nthread=8,seed=20)
    model.n_class = 2
    print("running grid search ...")
    print("=" * 50, '\n')
    searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
    searcher.fit(d_x, d_y)
    print(searcher.grid_scores_)
    print("=" * 50, '\n')
    print(searcher.best_params_)

if __name__ == '__main__':
    start = time.time()
    online = False
    print('loading data ...')
    train_data = pd.read_csv("round1_ijcai_18_train_20180301.txt", sep=" ")
    test_data = pd.read_csv("round1_ijcai_18_test_a_20180301.txt", sep=" ")
    is_test_in_train = test_data.user_id.isin(train_data.user_id)
    print("test在train中的个数为:{}".format(is_test_in_train.sum()))
    print("loading success ...")
    #缺失值用中位数替换
    none_list = ['item_brand_id','item_city_id','item_sales_level','user_gender_id','shop_review_positive_rate','shop_score_service','shop_score_delivery','shop_score_description']
    for label in none_list:
        test_data = test_data.replace({label: -1},test_data[label].mean())#mean()
        train_data = train_data.replace({label:-1},train_data[label].mean())
    train_data.drop_duplicates(inplace=True)#默认为所有行相同去重，也可以指定列下去重
    train_data = covert_seconds_to_days(train_data)
    #print(train_data.columns)
    #print(train_data.day.value_counts())
    
    features = ['context_id',
             'context_page_id',
             'instance_id',
             'is_trade',
             'item_brand_id',             
             'item_city_id',
             'item_collected_level',
             'item_id',
             'item_price_level',
             'item_pv_level',
             'shop_id',
             'shop_review_positive_rate',
             'shop_score_delivery',
             'shop_score_description',
             'shop_star_level',
             'user_age_level',
             'user_gender_id',
             'user_id',
             'day',
             'hour']
    labels = 'is_trade'
    lgb_baselinemodel(train_data,test_data,online)
    end1 = time.time()
    print("耗时：", (end1 - start))
    print('model-1 over')
    
#     lgb_gridsearch_model(train_data)
#     end2 = time.time()
#     print("耗时:", (end2 - start))
#     print('model-2 over')
                            

loading data ...
test在train中的个数为:5173
loading success ...


C:\Users\dell\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['user_gender_id']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


0.0831616170653
耗时： 11.616224527359009
model-1 over


C:\Users\dell\Anaconda3\lib\site-packages\ipykernel\__main__.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
import pandas as pd
# import numpy as np
import lightgbm as lgb
import time
from sklearn.metrics import log_loss


# from sklearn.model_selection import train_test_split,GridSearchCV


def timestamp_convert(value):
    format = '%Y-%m-%d %H:%M:%S'  # 要转化的格式
    value = time.localtime(value)
    # print(value)
    newdata = time.strftime(format, value)
    return newdata


def covert_seconds_to_days(train_data):
    # frame['days'] = (pd.to_datetime(frame['date']) - parse('2015-01-01')).dt.days#转化为时间
    train_data['date'] = train_data['context_timestamp'].apply(timestamp_convert)  # 将这一列的每个数据进行一一转换
    train_data['day'] = train_data.date.apply(lambda x: int(x[8:10]))  # 将这一列的每个数据进行一一切片
    train_data['hour'] = train_data.date.apply(lambda x: int(x[11:13]))
    #     user_query_day = data.groupby(['user_id', 'day']).size().reset_index().rename(columns={0:'user_query_day'})
    #      #默认inner。left、right分别保留左边、右边部分数据（缺失补NAN）,inner公共部分,outer俩边所有
    #     data = pd.merge(data, user_query_day, how='inner', on=['user_id','user_query_day'])
    #     user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index().rename(columns={1:"user_query_day_hour"})
    #     data = pd.merge(data, user_query_day_hour, how='inner', on=['user_id', 'day', 'hour']
    return train_data

def lgb_baselinemodel(train_data, test_data, online):

    #print(train_data.columns.tolist())#去掉  is_trade
    features = ['instance_id', 'item_id', 'item_brand_id', 'item_city_id', 'user_id', 'user_age_level',
                'user_occupation_id', 'user_star_level', 'context_id', 'context_page_id', 'shop_id',
                'shop_review_num_level', 'day', 'hour', 'gender_class_0', 'gender_class_2',
                'item_pv_sales', 'item_pv_collected', 'item_total_sales', 'shop_score', 'shop_view']

    label = ['is_trade']

    if online == False:
        training_data = train_data[(train_data['day'] < 24) & (train_data['day'] >= 18)]
        train_test_data = train_data[train_data['day'] == 24]
        clf = lgb.LGBMClassifier(num_leaves=64, max_depth=8, n_estimators=100, n_jobs=20)
        clf.fit(training_data[features], training_data[label], feature_name=features)
        train_test_data['pred'] = clf.predict_proba(train_test_data[features], )[:, 1]
        logloss = log_loss(train_test_data['is_trade'], train_test_data['pred'])
        print('logloss value：{}'.format(logloss))


# if online == True:
#        train_data = train_data.copy()
#        test_data = pd.read_csv("round1_ijcai_18_test_a_20180301.txt", sep=" ")
#        test_data.drop_duplicates(inplace=True)
#        test_data = covert_seconds_to_days(test_data)
#        model = lgb.LGBMClassifier(num_leaves=64, max_depth=7, n_estimators=80, n_jobs=20)
#        model.fit(train_data[features], train_data[label], feature_name=features)
#        test_data['proba'] = model.predict_proba(test_data[features],)[:, 1]
#        test_data[['instance_id','proba']].to_csv('baseline_lgb.csv', index=False, sep=" ")

def main():
    print('loading data ...')
    train_data = pd.read_csv("round1_ijcai_18_train_20180301.txt", sep=" ")
    test_data = pd.read_csv("round1_ijcai_18_test_a_20180301.txt", sep=" ")
    print("loading success ...")
    print("dealing with NAN")
    nan_list = ['item_brand_id', 'item_city_id', 'item_sales_level', 'user_age_level', 'user_occupation_id',
                'user_star_level',
                'user_gender_id', 'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery',
                'shop_score_description']
    # 缺失值-1替换为中位数
    for label in nan_list:
        train_data = train_data.replace({label: -1}, train_data[label].median())  # mean()
        train_data = train_data.replace({label: -1}, train_data[label].median())
    for label in nan_list:
        test_data = test_data.replace({label: -1}, test_data[label].median())  # mean()
        test_data = test_data.replace({label: -1}, test_data[label].median())

    train_data.drop_duplicates(inplace=True)  # 默认为所有行相同去重，也可以指定列下去重
    train_data = covert_seconds_to_days(train_data)
    train_data.drop(['date', 'context_timestamp'], axis=1, inplace=True)
    # 先不分析属性从属关系
    train_data.drop(['item_category_list', 'item_property_list', 'predict_category_property'], axis=1, inplace=True)

    # 归一化
    # train_data.context_page_id = train_data.context_page_id.values - 4000
    # train_data.user_age_level = train_data.user_age_level.values - 1000
    train_data.context_page_id = (train_data['context_page_id'] - train_data['context_page_id'].min())/(train_data['context_page_id'].max() - train_data['context_page_id'].min())
    train_data.user_age_level = (train_data['user_age_level'] - train_data['user_age_level'].min())/(train_data['user_age_level'].max() - train_data[    'user_age_level'].min())
    # 将user_gender_id属性分割成多个(适合比较少类目的属性)
    gender_id_class = pd.get_dummies(train_data.user_gender_id)
    gender_class = gender_id_class.rename(columns=lambda x: 'gender_class_' + str(x))
    train_data = pd.concat([train_data, gender_class], axis=1)
    train_data.drop(['user_gender_id'], axis=1, inplace=True)

    #item内构建新特征
    train_data['item_pv_sales'] = train_data['item_pv_level'] / train_data['item_sales_level']
    train_data['item_pv_collected'] = train_data['item_pv_level'] / train_data['item_collected_level']
    train_data['item_total_sales'] = train_data['item_price_level'] * train_data['item_sales_level']
    train_data.drop(['item_pv_level','item_sales_level','item_collected_level','item_price_level'],
                    axis=1,inplace=True)

    #shop内构建特征
    train_data['shop_score'] = (train_data.shop_score_service * train_data.shop_score_delivery * train_data.shop_score_description) / 3
    train_data['shop_view'] = train_data.shop_review_positive_rate * train_data.shop_star_level
    train_data.drop(['shop_score_service', 'shop_score_delivery', 'shop_score_description'],
                    axis=1, inplace=True)
    train_data.drop(['shop_review_positive_rate','shop_star_level'],
                    axis=1,inplace=True)
#     dfdata = train_data.corr()   
#     plt.subplots(figsize=(30,30))
#     #vmax,vmin:分别是热力图的颜色取值最大和最小范围
#     #annot(annotate的缩写):默认取值False；如果是True，在热力图每个方格写入数据
#     sns.heatmap(dfdata, annot=True, vmax=1, square=True, cmap='Reds')
#     plt.savefig("./features_corr.png")
#     plt.show()

    lgb_baselinemodel(train_data, test_data, online)


if __name__ == '__main__':
    start = time.time()
    online = False
    main()
    print("耗时：{}".format((time.time() - start)))

loading data ...
loading success ...
dealing with NAN


C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


logloss value：0.08310006154955947
耗时：15.208750009536743


C:\Users\dell\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
